## Solving second order ODEs as coupled first order ODEs

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

For a simple harmonic oscillator (SHO) we have the basic equation:
$$m\frac{d^{2}x}{dt^{2}} = -kx$$

Using the standard definition of velocity:
$$v = \frac{dx}{dt}$$ 

we can rewrite the SHO equation:
$$\frac{dv}{dt} = -\frac{kx}{m}$$ 

This now gives us *two coupled* equations, both of which are first order ODEs.  We must solve them *together* since they are coupled.  Note that we normally write the first equation as:

$$\frac{dx}{dt} = v$$

so that we have the derivative on the left, and its definition on the right, to be compatible with the previous equation.

Expanding the derivatives using a simple forward finite difference, $dx/dt = \left(x(t+\Delta t) - x(t)\right)/\Delta t$, we can rewrite the equations:
$$x(t+\Delta t) = x(t) + v(t)\times \Delta t$$
$$v(t+\Delta t) = v(t) - \frac{kx(t)}{m}\times \Delta t$$

Note that this simple expansion gives us Euler's method, which we know is low accuracy (first order in $\Delta t$).  I will use it for simplicity below, though it's too inaccurate for serious analysis.

In [ ]:
# SHO parameters (assume SI units throughout)
m = 1.0
k = 1.0
dt = 0.01
total_t = 10
N_steps = int(total_t/dt)+1
time = np.linspace(0,total_t,N_steps)

In [ ]:
# Initial conditions
x0 = 1.0
v0 = 0.0

# Storage
x = np.zeros(N_steps)
v = np.zeros(N_steps)

# Iterate over the coupled equations: this is Euler's method
# which we know is not accurate, but is clear
x[0] = x0
v[0] = v0
for i in range(0,N_steps-1):
    x[i+1] = x[i] + v[i] * dt
    v[i+1] = v[i] - (k*x[i]/m) * dt

In [ ]:
plt.plot(time,x)
plt.xlabel('Time')
plt.ylabel('Displacement')
plt.title(f'Motion of SHO from Euler with dt={dt}')
plt.grid()

The code above was a direct translation of the equations, but is not reusable for other problems, and is not efficient.  Now we turn to a case where we combine $x$ and $v$ into a single array as if we were solving a vector first order ODE.

To do this, we have to combine the right-hand sides of the coupled ODEs into a single array, using a routine like the one below:

In [ ]:
def SHO_deriv(y,t):
    """Evaluate derivatives for position and velocity for SHO
    with both quantities in same array
    
    Inputs:
    y  Array containing x and v at present timestep
    t  Time (not used here but required for general problem)
    
    Output:
    dy Array containing dx and dv
    """
    x = y[0]
    v = y[1]
    dx = v
    dv = -k*x/m
    return np.array([dx,dv])

Now we can propagate any number of coupled equations with the code below; the only thing that would change between implementations would be the dimensions of`y` and `y0` and the return value of the right-hand side function (`SHO_deriv` in this case).  Note that these have to be the same!

In [ ]:
# Set up array to store both position and velocity for all time
y = np.zeros((N_steps,2))

# Initialise and run Euler's method with a simple update step
y[0] = np.array([x0,v0])
for i in range(0,N_steps-1):
    y[i+1] = y[i] + SHO_deriv(y[i],time[i]) * dt

In [ ]:
# Plot resulting displacement
plt.plot(time,y[:,0])
plt.xlabel('Time')
plt.ylabel("Displacement")
plt.title(f"Motion of SHO from Euler, with dt={dt}")
plt.grid()

Finally, we extend this solution to a problem in two dimensions, so that position and velocity now have two components.  I will set up the initial velocity so that we should end up with circular motion; note that both components are just undergoing simple harmonic motion.

As before, the only change here is in the dimensions of `y2d` and the initial position and velocity; because of the way we wrote the function, it needs no update.

In [ ]:
# Set up position and velocity in two dimensions
x2d0 = np.array((1.0,0.0))
v2d0 = np.array((0.0,1.0))

# Care needed with indexing: timesteps, then position/velocity, then x/y
y2d = np.zeros((N_steps,2,2))

# And iterate exactly as before (no change needed)
y2d[0] = np.array([x2d0,v2d0])
for i in range(0,N_steps-1):
    y2d[i+1] = y2d[i] + SHO_deriv(y2d[i],time[i]) * dt

In [ ]:
# Plot x vs y for all timesteps to see overall motion
plt.plot(y2d[:,0,0],y2d[:,0,1]) # All timesteps, position, x and y
plt.axis('scaled')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Trajectory')

In [ ]:
plt.plot(time,y2d[:,0,0],label='x')
plt.plot(time,y2d[:,0,1],label='y')
plt.xlabel('Time')
plt.ylabel("Displacement")
plt.title(f"Motion of 2D SHO from Euler, with dt={dt}")
plt.legend()
plt.grid()